# Course Project: Galaxy Classifier

### Lexy Andershock, Gian Fernandez-Aleman, David Long

In [22]:
# Import required libraries
import h5py
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm
from tensorflow.keras import utils
from matplotlib.colors import LinearSegmentedColormap

# Scikit-learn libraries and routines
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from scipy.optimize import minimize
from sklearn.multiclass import OneVsRestClassifier

In [23]:
# To get the images and labels from file
with h5py.File('Galaxy10.h5', 'r') as data:
    images = np.array(data['images'])
    labels = np.array(data['ans'])

# To convert the labels to categorical 10 classes
labels = utils.to_categorical(labels, 10)

# To convert to desirable type
labels = labels.astype(np.float32)
images = images.astype(np.float32)

In [24]:
# Split the data into training and test sets
train_idx, test_idx = train_test_split(np.arange(labels.shape[0]), test_size=0.1)
train_images, train_labels, test_images, test_labels = images[train_idx], labels[train_idx], images[test_idx], labels[test_idx]

In [ ]:
# Because we are dealing with image data and we are working with logistic
# regression, a linear model, the images will have to be flattened.
n_samples, height, width, channels = train_images.shape
train_images_flat = train_images.reshape(n_samples, -1)
test_images_flat = test_images.reshape(test_images.shape[0], -1)

# Standardizing the data will allow logistic regression to perform better
scaler = StandardScaler()
train_images_scaled = scaler.fit_transform(train_images_flat)
test_images_scaled = scaler.transform(test_images_flat)

log_reg = OneVsRestClassifier(LogisticRegression(max_iter=5000, multi_class='ovr', solver='saga'))

log_reg.fit(train_images_scaled, np.argmax(train_labels, axis=1))

predictions = log_reg.predict(test_images_scaled)

cm = confusion_matrix(np.argmax(test_labels, axis=1), predictions)
disp = ConfusionMatrixDisplay(confusion_matrix=cm)
disp.plot(cmap=plt.cm.Blues)
plt.show()

print(classification_report(np.argmax(test_labels, axis=1), predictions))

c:\Users\longd\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:1256: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. Use OneVsRestClassifier(LogisticRegression(..)) instead. Leave it to its default value to avoid this warning.
  warnings.warn(
